# End of game measures
This `Python` notebook takes the anonymized data and computes population-level measures for each game.

In [3]:
%pylab inline
import json
import numpy as np
import pandas as pd
import glob
import itertools
from sklearn.decomposition import PCA
from scipy import stats

from helpers import shuffle

Populating the interactive namespace from numpy and matplotlib


In [4]:
output_dir = "../results-anonymized/pilot/"
files = glob.glob(output_dir+'block_*_pilot.json')
files

['../results-anonymized/pilot/block_20200505_pilot.json',
 '../results-anonymized/pilot/block_20200507_pilot.json',
 '../results-anonymized/pilot/block_20200624_pilot.json',
 '../results-anonymized/pilot/block_20200626_pilot.json',
 '../results-anonymized/pilot/block_20200506_pilot.json']

In [5]:
blocks = []
for file in files:
    with open(file) as f:
        blocks.append(json.load(f))

In [6]:
# Enumerate clues to be used in polarization analysis
# final clues used in analysis are connections between hub nodes (1,2) and rim nodes (3-13)
t_spokes = ['tclue_1_3', 'tclue_1_4', 'tclue_1_5', 'tclue_1_6', 'tclue_1_7', 
            'tclue_1_8', 'tclue_1_9', 'tclue_1_10','tclue_1_11', 'tclue_1_12', 'tclue_1_13',
            'tclue_2_3', 'tclue_2_4', 'tclue_2_5', 'tclue_2_6', 'tclue_2_7',
            'tclue_2_8', 'tclue_2_9', 'tclue_2_10', 'tclue_2_11', 'tclue_2_12', 'tclue_2_13']

c_spokes = ['cclue_1_3', 'cclue_1_4', 'cclue_1_5', 'cclue_1_6', 'cclue_1_7', 
            'cclue_1_8', 'cclue_1_9', 'cclue_1_10','cclue_1_11', 'cclue_1_12', 'cclue_1_13',
            'cclue_2_3', 'cclue_2_4', 'cclue_2_5', 'cclue_2_6', 'cclue_2_7',
            'cclue_2_8', 'cclue_2_9', 'cclue_2_10', 'cclue_2_11', 'cclue_2_12', 'cclue_2_13']

# Enumerate end-of-game survey questions to be used in polarization analysis
assessments = ['appearance_1', 'appearance_2', 
               'clothing_1', 'clothing_2',
               'suspect_1', 'suspect_2', 'suspect_3',
               'tool_1', 'tool_2', 
               'vehicle_1', 'vehicle_2']    
    
def compute_single_point_measures(game):
    """ 
    Compute the game-level measures 
    
    "Games" in this experiment contain both a treatment and control condition
    and these must be properly separated from one another.
    
    """
    # Form end-of-game survey responses into a dataframe
    collector = {}
    for p, k in game['players'].items():
        try:
            collector[k['data.position']] = k['data.caseMade']
        except:
            print('%s did not complete the post-game survey' %k['data.position'])
    responses = pd.DataFrame(collector).T.sort_index()

    # Form final notebook states into a dataframe
    final_adoptions = pd.DataFrame(data=0, index=responses.index, columns=t_spokes+c_spokes)
    for p, k in game['players'].items():
        for clue_id in k['data.notebooks']['promising_leads']['clueIDs']:
            final_adoptions.loc[k['data.position'], clue_id] = 1


    # Determine the number of datapoints to be used in polarization analysis
    # if there are missing responses, need to compare equal sized datasets
    t_responses = [pos for pos in responses.index if pos.startswith('t')]
    c_responses = [pos for pos in responses.index if pos.startswith('c')] 
    # use whichever condition has fewer responses to set the sample size
    n_used = min(len(t_responses), len(c_responses))


    def process_subset(subset, spokes):
        """ compute a result on the selected subset of the data """
        sub_res = {}
        
        # select the subset of the survey responses that will be used in the subset analysis
        sub_survey = responses.loc[subset, assessments]
        
        # survey PC1 
        pca = PCA(n_components=1)
        pca.fit(sub_survey)  
        sub_res['survey PC1'] = pca.explained_variance_ratio_

        # survey similarity percentiles
        survey_corrs = sub_survey.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['survey 5% similarity'], sub_res['survey 95% similarity'] = np.percentile(
            survey_corrs, [5, 95])
        
        # select the subset of the behavioral responses that will be used in the subset analysis
        sub_adopt = final_adoptions.loc[subset, spokes]
        
        # final-state PC1
        pca = PCA(n_components=1)
        pca.fit(sub_adopt)  
        sub_res['spoke PC1'] = pca.explained_variance_ratio_
        
        # final state similarity percentiles
        spoke_corrs = sub_adopt.T.corr().mask(np.tri(n_used, n_used, 0, dtype='bool')).stack()
        sub_res['spoke 5% similarity'], sub_res['spoke 95% similarity'] = np.percentile(
            spoke_corrs, [5, 95])
        
        # compute the expected values for the given level of adoption
        # by shuffling the clues between individuals 
        # (preserving the number of clues each individual holds, 
        # and the number of individuals holding each clue)
        # do this a number of times and average the result
        e95 = []
        e5 = []
        ePC1 = []
        for _ in range(100):
            shuffle_adopt = pd.DataFrame(index=sub_adopt.index,
                                         columns=sub_adopt.columns,
                                         data=shuffle(sub_adopt.values, n=500))

            n_agents = len(shuffle_adopt.index)
            corrs = shuffle_adopt.astype(float).T.corr().mask(np.tri(n_agents, n_agents, 0, dtype='bool')).stack()
            e95.append(np.percentile(corrs, 95))
            e5.append(np.percentile(corrs, 5))

            pca = PCA(n_components=1)
            pca.fit(shuffle_adopt)
            ePC1.append(pca.explained_variance_ratio_[0])
        
        # compute the net effect of (interdependent or independent) diffusion 
        # over chance distribution of the same clues
        sub_res['net spoke PC1'] = sub_res['spoke PC1'] - np.mean(ePC1)
        sub_res['net spoke 95% similarity'] = sub_res['spoke 95% similarity'] - np.mean(e95)
        sub_res['net spoke 5% similarity'] = sub_res['spoke 5% similarity'] - np.mean(e5)
        
        return sub_res
        
        
    # For each subset of size 'n_used', compute a result. 
    # In most cases there are no missing responses, so just compute on the complete set
    t_collector = []
    for subset in itertools.combinations(t_responses, r=n_used):
        t_collector.append(process_subset(subset, t_spokes))

    # The recorded result is the average over all subsets
    if len(t_collector) > 1:
        print('Averaging over %i combinations for treatment case'%len(t_collector))
    t_result = pd.DataFrame(t_collector).mean()

    # Compute average for confidence and consensus measures on all submissions
    t_result['confidence'] = responses.loc[t_responses, 'confidence'].mean()
    t_result['consensus'] = responses.loc[t_responses, 'consensus'].mean()    
    t_result['id'] = game['_id']
    
    # Perform the same analysis as above for the control condition
    c_collector = []
    for subset in itertools.combinations(c_responses, r=n_used):
        c_collector.append(process_subset(subset, c_spokes))

    if len(c_collector) > 1:
        print('Averaging over %i combinations for control case'%len(c_collector))
    c_result = pd.DataFrame(c_collector).mean()

    c_result['confidence'] = responses.loc[c_responses, 'confidence'].mean()
    c_result['consensus'] = responses.loc[c_responses, 'consensus'].mean()     

    c_result['id'] = game['_id']
    
    #pd.merge(t_result, c)result, suffixes=(' (inter)', ' (indep)'))
    result = pd.concat([t_result, c_result], keys=['inter', 'indep'])
    #result['game_id']=game['createdAt'].split('_')[0].replace('-','_').replace(':','_').replace('.','_')
    return result

def compute_block(block):
    results_collector = []
    network_collector = []
    for name, game in block.items():
        network_collector.append('caveman' if 'caveman' in name else 'dodec')
        results_collector.append(compute_single_point_measures(game))
        
    result = pd.concat(results_collector, keys=network_collector)
    return result

In [7]:
measurements = pd.concat([compute_block(block) for block in blocks], axis=1)
#measurements.to_csv(output_dir+"end_of_game_measurements.csv")
measurements

0                  1  \
caveman indep confidence                             53.4              57.55   
              consensus                             55.15              54.55   
              id                        Bd3w7KBP8ZKKMBxhj  5wifJzSKTJuDtQkzS   
              net spoke 5% similarity           -0.183997          -0.204713   
              net spoke 95% similarity           0.315044            0.31569   
              net spoke PC1                       0.20543           0.240604   
              spoke 5% similarity                -0.41833          -0.313527   
              spoke 95% similarity               0.806832           0.900423   
              spoke PC1                          0.382635           0.422482   
              survey 5% similarity              -0.725347          -0.621686   
              survey 95% similarity              0.747789           0.723486   
              survey PC1                         0.359687           0.440281   
        inter confidence                             53.5              52.35   
              consensus                              51.6               57.1   
              id                        Bd3w7KBP8ZKKMBxhj  5wifJzSKTJuDtQkzS   
              net spoke 5% similarity           -0.137348           -0.17313   
              net spoke 95% similarity              0.376           0.297532   
              net spoke PC1                      0.316696           0.213024   
              spoke 5% similarity               -0.340905          -0.430331   
              spoke 95% similarity               0.896421           0.783046   
              spoke PC1                            0.5056           0.397125   
              survey 5% similarity              -0.553209          -0.539193   
              survey 95% similarity              0.873783           0.686564   
              survey PC1                         0.488426           0.354293   
dodec   indep confidence                            43.35               50.5   
              consensus                              42.7               50.6   
              id                        xXvsHQvoQC4LsoyCi  acKghgvvb3kewvMd9   
              net spoke 5% similarity           -0.171183           -0.13768   
              net spoke 95% similarity           0.196308           0.144668   
              net spoke PC1                     0.0870114           0.115307   
              spoke 5% similarity               -0.293518          -0.318754   
              spoke 95% similarity                0.71743            0.67082   
              spoke PC1                          0.342774           0.374724   
              survey 5% similarity              -0.492309          -0.485883   
              survey 95% similarity               0.57503           0.739458   
              survey PC1                         0.279186           0.419603   
        inter confidence                            59.35               58.3   
              consensus                              53.5              54.45   
              id                        xXvsHQvoQC4LsoyCi  acKghgvvb3kewvMd9   
              net spoke 5% similarity           -0.151974          -0.127196   
              net spoke 95% similarity           0.178643         0.00882393   
              net spoke PC1                      0.114433          0.0568829   
              spoke 5% similarity               -0.334691          -0.138534   
              spoke 95% similarity               0.716328           0.642071   
              spoke PC1                          0.301137           0.249915   
              survey 5% similarity              -0.549573          -0.351835   
              survey 95% similarity              0.746668           0.690122   
              survey PC1                         0.417522           0.311937   

                                                        2                  3  \
caveman indep confidence                              N

In [8]:
# write a flat output file for the games, to be used in mediation analysis
measurements_flat = measurements.unstack(level=[0,1])
measurements_flat.loc['t'] = [1 if c[2]=="inter" else 0 for c in measurements_flat.columns]
measurements_flat.loc['network'] = [1 if c[1]=="caveman" else 0 for c in measurements_flat.columns]
measurements_flat.columns = ["GT_" + str(measurements_flat.loc['id'][c]) + ("_1" if c[2]=="inter" else "_0") for c in measurements_flat.columns]
measurements_flat.T.to_csv(output_dir+"end_of_game_measurements.csv")
measurements_flat.T

,confidence,consensus,id,net spoke 5% similarity,net spoke 95% similarity,net spoke PC1,spoke 5% similarity,spoke 95% similarity,spoke PC1,survey 5% similarity,survey 95% similarity,survey PC1,t,network
GT_Bd3w7KBP8ZKKMBxhj_0,53.4,55.15,Bd3w7KBP8ZKKMBxhj,-0.183997,0.315044,0.20543,-0.41833,0.806832,0.382635,-0.725347,0.747789,0.359687,0,1
GT_Bd3w7KBP8ZKKMBxhj_1,53.5,51.6,Bd3w7KBP8ZKKMBxhj,-0.137348,0.376,0.316696,-0.340905,0.896421,0.5056,-0.553209,0.873783,0.488426,1,1
GT_xXvsHQvoQC4LsoyCi_0,43.35,42.7,xXvsHQvoQC4LsoyCi,-0.171183,0.196308,0.0870114,-0.293518,0.71743,0.342774,-0.492309,0.57503,0.279186,0,0
GT_xXvsHQvoQC4LsoyCi_1,59.35,53.5,xXvsHQvoQC4LsoyCi,-0.151974,0.178643,0.114433,-0.334691,0.716328,0.301137,-0.549573,0.746668,0.417522,1,0
GT_5wifJzSKTJuDtQkzS_0,57.55,54.55,5wifJzSKTJuDtQkzS,-0.204713,0.31569,0.240604,-0.313527,0.900423,0.422482,-0.621686,0.723486,0.440281,0,1
GT_5wifJzSKTJuDtQkzS_1,52.35,57.1,5wifJzSKTJuDtQkzS,-0.17313,0.297532,0.213024,-0.430331,0.783046,0.397125,-0.539193,0.686564,0.354293,1,1
GT_acKghgvvb3kewvMd9_0,50.5,50.6,acKghgvvb3kewvMd9,-0.13768,0.144668,0.115307,-0.318754,0.67082,0.374724,-0.485883,0.739458,0.419603,0,0
GT_acKghgvvb3kewvMd9_1,58.3,54.45,acKghgvvb3kewvMd9,-0.127196,0.00882393,0.0568829,-0.138534,0.642071,0.249915,-0.351835,0.690122,0.311937,1,0
GT_nan_0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1
GT_nan_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [9]:
# todo: drop id row for subsequent analysis
measurements.drop('id', level=2, inplace=True)
measurements.index = measurements.index.remove_unused_levels()

In [58]:
def bootstrap_mean(l, q=(2.5, 97.5), n=1000):
    "Basic bootstrap confidence intervals (q) with n resamples"
    return np.percentile([np.mean(np.random.choice(l, size=len(l))) for _ in range(n)], q=q)


def mean_result(measure1, _):
    return measure1.mean()

def mean_lowerbound(measure1, _):
    return bootstrap_mean(measure1)[0]

def mean_upperbound(measure1, _):
    return bootstrap_mean(measure1)[1]

def effect_size(measure1, measure2):
    return (measure1 - measure2).mean()

def effect_lowerbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[0]

def effect_upperbound(measure1, measure2):
    return bootstrap_mean(measure1 - measure2)[1]

def effect_p_val(measure1, measure2):
    return stats.ttest_rel(measure1, measure2)[1]

def effect_p_val_one_tail(measure1, measure2):
    return stats.ttest_rel(measure1, measure2)[1]/2

def make_table(measurements, func, network='dodec', treatment='indep'):
    rows = measurements.index.levels[2]
    cols = np.unique(measurements.index.droplevel(2))
    res = pd.DataFrame(index=rows, columns=cols)
    for row in rows:
        for col in cols:
            comparison = measurements.loc[col+tuple([row])].dropna()
            #reference = measurements.loc[('dodec', 'indep')+tuple([row])][comparison.index]
            reference = measurements.loc[(network, treatment)+tuple([row])][comparison.index]
            res.at[row, col] = func(comparison, reference)
    return res

In [45]:
res = make_table(measurements, mean_result)
res

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,52.2833,56.7667,48.99,53.38
consensus,52.7667,58.3333,49.07,53.02
net spoke 5% similarity,-0.173295,-0.121702,-0.066612,-0.0970896
net spoke 95% similarity,0.285114,0.382026,0.11777,0.0967325
net spoke PC1,0.201012,0.247279,0.0582617,0.0752066
spoke 5% similarity,-0.342128,-0.353304,-0.177564,-0.245504
spoke 95% similarity,0.83705,0.893156,0.698852,0.646924
spoke PC1,0.404108,0.43493,0.299288,0.269057
survey 5% similarity,-0.6126,-0.556754,-0.395819,-0.469243
survey 95% similarity,0.729252,0.788166,0.711058,0.683996


In [46]:
res_lowerbound = make_table(measurements, mean_lowerbound)
res_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,45.9,52.35,45.56,48.43
consensus,48.6,51.6,44.06,51.51
net spoke 5% similarity,-0.204713,-0.17313,-0.131176,-0.13778
net spoke 95% similarity,0.224607,0.297532,0.0729009,0.0439875
net spoke PC1,0.157001,0.212116,0.0255982,0.0519753
spoke 5% similarity,-0.41833,-0.430331,-0.276488,-0.324874
spoke 95% similarity,0.803894,0.783046,0.647441,0.618184
spoke PC1,0.382635,0.397125,0.253144,0.253164
survey 5% similarity,-0.725347,-0.57786,-0.505146,-0.540434
survey 95% similarity,0.71648,0.686564,0.639585,0.6477


In [47]:
res_upperbound = make_table(measurements, mean_upperbound)
res_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,57.55,64.45,52.64,58.21
consensus,55.15,66.3,54.08,54.06
net spoke 5% similarity,-0.131175,-0.0546296,-0.002048,-0.0563988
net spoke 95% similarity,0.31569,0.472547,0.162603,0.147078
net spoke PC1,0.229173,0.316696,0.0943166,0.0978519
spoke 5% similarity,-0.294525,-0.288675,-0.0696658,-0.166135
spoke 95% similarity,0.900423,1,0.758169,0.683866
spoke PC1,0.422482,0.5056,0.345432,0.288877
survey 5% similarity,-0.490766,-0.539193,-0.275828,-0.39533
survey 95% similarity,0.747789,0.873783,0.760302,0.721845


In [48]:
eff = make_table(measurements, effect_size)
eff

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,2.4,6.88333,0,4.39
consensus,3.15,8.71667,0,3.95
net spoke 5% similarity,-0.0576224,-0.0060301,0,-0.0304776
net spoke 95% similarity,0.127767,0.224679,0,-0.0210372
net spoke PC1,0.133455,0.179722,0,0.0169449
spoke 5% similarity,-0.14851,-0.159686,0,-0.0679407
spoke 95% similarity,0.106442,0.162549,0,-0.0519284
spoke PC1,0.0675537,0.0983758,0,-0.030231
survey 5% similarity,-0.198728,-0.142882,0,-0.0734243
survey 95% similarity,0.0466303,0.105545,0,-0.0270628


In [59]:
eff_pval = make_table(measurements.dropna(axis=1), effect_p_val_one_tail)
eff_pval

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0.368207,0.0572389,NaN,0.114831
consensus,0.315621,0.00967246,NaN,0.287814
net spoke 5% similarity,0.067461,0.398991,NaN,0.236889
net spoke 95% similarity,0.0152231,0.0311936,NaN,0.085688
net spoke PC1,0.00384175,0.0245112,NaN,0.388029
spoke 5% similarity,0.131548,0.0959673,NaN,0.435208
spoke 95% similarity,0.125877,0.0120126,NaN,0.1712
spoke PC1,0.0526717,0.0691024,NaN,0.0718296
survey 5% similarity,0.0121119,0.118895,NaN,0.423438
survey 95% similarity,0.268335,0.206609,NaN,0.34962


In [50]:
eff_lowerbound = make_table(measurements, effect_lowerbound)
eff_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,-9.9,1.85,0,-0.73
consensus,-6.95,6.5,0,-1.674
net spoke 5% similarity,-0.0930209,-0.0354497,0,-0.0925387
net spoke 95% similarity,0.0935412,0.152865,0,-0.0831234
net spoke PC1,0.118418,0.0977169,0,-0.0240861
spoke 5% similarity,-0.325946,-0.320095,0,-0.180523
spoke 95% similarity,0.00032269,0.112226,0,-0.126495
spoke PC1,0.039861,0.022401,0,-0.0892043
survey 5% similarity,-0.233037,-0.314437,0,-0.186426
survey 95% similarity,-0.0168953,-0.0528939,0,-0.103573


In [51]:
eff_upperbound = make_table(measurements, effect_upperbound)
eff_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,10.05,10.15,0,10.93
consensus,12.45,10.75,0,9.57
net spoke 5% similarity,-0.0128135,0.0338352,0,0.0102082
net spoke 95% similarity,0.171023,0.341481,0,0.0408899
net spoke PC1,0.156649,0.229685,0,0.0499939
spoke 5% similarity,0.00522697,-0.0473868,0,0.0598554
spoke 95% similarity,0.229602,0.196429,0,-0.00371858
spoke PC1,0.115042,0.162826,0,0.0222034
survey 5% similarity,-0.135803,-0.0533101,0,0.0433194
survey 95% similarity,0.172759,0.298753,0,0.073216


In [52]:
export_table = pd.DataFrame()
export_table["Result Mean"] = res.unstack()
export_table["Result Lower Bound"] = res_lowerbound.unstack()
export_table["Result Upper Bound"] = res_upperbound.unstack()
export_table["Effect Size"] = eff.unstack()
export_table["Effect Upper Bound"] = eff_upperbound.unstack()
export_table["Effect Lower Bound"] = eff_lowerbound.unstack()
export_table["Effect P Value"] = eff_pval.unstack()


export_table.to_csv(output_dir+"end_of_game_effects_summary.csv")
export_table

Result Mean Result Lower Bound  \
(caveman, indep) confidence                   52.2833               45.9   
                 consensus                    52.7667               48.6   
                 net spoke 5% similarity    -0.173295          -0.204713   
                 net spoke 95% similarity    0.285114           0.224607   
                 net spoke PC1               0.201012           0.157001   
                 spoke 5% similarity        -0.342128           -0.41833   
                 spoke 95% similarity         0.83705           0.803894   
                 spoke PC1                   0.404108           0.382635   
                 survey 5% similarity         -0.6126          -0.725347   
                 survey 95% similarity       0.729252            0.71648   
                 survey PC1                  0.368261           0.304816   
(caveman, inter) confidence                   56.7667              52.35   
                 consensus                    58.3333               51.6   
                 net spoke 5% similarity    -0.121702           -0.17313   
                 net spoke 95% similarity    0.382026           0.297532   
                 net spoke PC1               0.247279           0.212116   
                 spoke 5% similarity        -0.353304          -0.430331   
                 spoke 95% similarity        0.893156           0.783046   
                 spoke PC1                    0.43493           0.397125   
                 survey 5% similarity       -0.556754           -0.57786   
                 survey 95% similarity       0.788166           0.686564   
                 survey PC1                  0.423766           0.354293   
(dodec, indep)   confidence                     48.99              45.56   
                 consensus                      49.07              44.06   
                 net spoke 5% similarity    -0.066612          -0.131176   
                 net spoke 95% similarity     0.11777          0.0729009   
                 net spoke PC1              0.0582617          0.0255982   
                 spoke 5% similarity        -0.177564          -0.276488   
                 spoke 95% similarity        0.698852           0.647441   
                 spoke PC1                   0.299288           0.253144   
                 survey 5% similarity       -0.395819          -0.505146   
                 survey 95% similarity       0.711058           0.639585   
                 survey PC1                  0.318089             0.2668   
(dodec, inter)   confidence                     53.38              48.43   
                 consensus                      53.02              51.51   
                 net spoke 5% similarity   -0.0970896           -0.13778   
                 net spoke 95% similarity   0.0967325          0.0439875   
                 net spoke PC1              0.0752066          0.0519753   
                 spoke 5% similarity        -0.245504          -0.324874   
                 spoke 95% similarity        0.646924           0.618184   
                 spoke PC1                   0.269057           0.253164   
                 survey 5% similarity       -0.469243          -0.540434   
                 survey 95% similarity       0.683996             0.6477   
                 survey PC1                  0.342388           0.302235   

                                          Result Upper Bound Effect Size  \
(caveman, indep) confidence                            57.55         2.4   
                 consensus                             55.15        3.15   
                 net spoke 5% similarity           -0.131175  -0.0576224   
                 net spoke 95% similarity            0.31569    0.127767   
                 net spoke PC1                      0.229173    0.133455   
                 spoke 5% similarity               -0.294525    -0.14851   
                 spoke 95% similarity               0.900423    0.106442   
             

# compare to the caveman baseline

In [53]:
eff = make_table(measurements, effect_size, network='caveman')
eff

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0,4.48333,-2.4,5.51667
consensus,0,5.56667,-3.15,-0.05
net spoke 5% similarity,0,0.0515923,0.0576224,0.0647382
net spoke 95% similarity,0,0.0969126,-0.127767,-0.199848
net spoke PC1,0,0.0462669,-0.133455,-0.121415
spoke 5% similarity,0,-0.0111764,0.14851,0.128123
spoke 95% similarity,0,0.0561064,-0.106442,-0.182293
spoke PC1,0,0.030822,-0.0675537,-0.134968
survey 5% similarity,0,0.0558455,0.198728,0.181561
survey 95% similarity,0,0.0589146,-0.0466303,-0.0152375


In [54]:
eff_pval = make_table(measurements.dropna(axis=1), effect_p_val, network='caveman')
eff_pval

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,NaN,0.596925,0.736415,0.171425
consensus,NaN,0.471155,0.631243,0.962356
net spoke 5% similarity,NaN,0.0597669,0.134922,0.0591649
net spoke 95% similarity,NaN,0.344224,0.0304463,0.0654732
net spoke PC1,NaN,0.369968,0.0076835,0.0599827
spoke 5% similarity,NaN,0.861981,0.263095,0.0399274
spoke 95% similarity,NaN,0.604128,0.251753,0.0654705
spoke PC1,NaN,0.57515,0.105343,0.0389914
survey 5% similarity,NaN,0.538984,0.0242238,0.0666787
survey 95% similarity,NaN,0.353639,0.536669,0.25065


In [55]:
eff_lowerbound = make_table(measurements, effect_lowerbound, network='caveman')
eff_lowerbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0,-5.2,NaN,NaN
consensus,0,-3.55,NaN,NaN
net spoke 5% similarity,0,0.031583,NaN,NaN
net spoke 95% similarity,0,-0.0181581,NaN,NaN
net spoke PC1,0,-0.0275799,NaN,NaN
spoke 5% similarity,0,-0.116804,NaN,NaN
spoke 95% similarity,0,-0.117376,NaN,NaN
spoke PC1,0,-0.0253574,NaN,NaN
survey 5% similarity,0,-0.0870937,NaN,NaN
survey 95% similarity,0,-0.0369214,NaN,NaN


In [56]:
eff_upperbound = make_table(measurements, effect_upperbound, network='caveman')
eff_upperbound

,"(caveman, indep)","(caveman, inter)","(dodec, indep)","(dodec, inter)"
confidence,0,18.55,NaN,NaN
consensus,0,17.7,NaN,NaN
net spoke 5% similarity,0,0.076545,NaN,NaN
net spoke 95% similarity,0,0.24794,NaN,NaN
net spoke PC1,0,0.111266,NaN,NaN
spoke 5% similarity,0,0.0774247,NaN,NaN
spoke 95% similarity,0,0.196106,NaN,NaN
spoke PC1,0,0.122964,NaN,NaN
survey 5% similarity,0,0.172137,NaN,NaN
survey 95% similarity,0,0.125994,NaN,NaN


In [57]:
export_table = pd.DataFrame()
export_table["Result Mean"] = res.unstack()
export_table["Result Lower Bound"] = res_lowerbound.unstack()
export_table["Result Upper Bound"] = res_upperbound.unstack()
export_table["Effect Size"] = eff.unstack()
export_table["Effect Upper Bound"] = eff_upperbound.unstack()
export_table["Effect Lower Bound"] = eff_lowerbound.unstack()
export_table["Effect P Value"] = eff_pval.unstack()


export_table.to_csv(output_dir+"end_of_game_effects_summary_caveman_baseline.csv")
export_table

Result Mean Result Lower Bound  \
(caveman, indep) confidence                   52.2833               45.9   
                 consensus                    52.7667               48.6   
                 net spoke 5% similarity    -0.173295          -0.204713   
                 net spoke 95% similarity    0.285114           0.224607   
                 net spoke PC1               0.201012           0.157001   
                 spoke 5% similarity        -0.342128           -0.41833   
                 spoke 95% similarity         0.83705           0.803894   
                 spoke PC1                   0.404108           0.382635   
                 survey 5% similarity         -0.6126          -0.725347   
                 survey 95% similarity       0.729252            0.71648   
                 survey PC1                  0.368261           0.304816   
(caveman, inter) confidence                   56.7667              52.35   
                 consensus                    58.3333               51.6   
                 net spoke 5% similarity    -0.121702           -0.17313   
                 net spoke 95% similarity    0.382026           0.297532   
                 net spoke PC1               0.247279           0.212116   
                 spoke 5% similarity        -0.353304          -0.430331   
                 spoke 95% similarity        0.893156           0.783046   
                 spoke PC1                    0.43493           0.397125   
                 survey 5% similarity       -0.556754           -0.57786   
                 survey 95% similarity       0.788166           0.686564   
                 survey PC1                  0.423766           0.354293   
(dodec, indep)   confidence                     48.99              45.56   
                 consensus                      49.07              44.06   
                 net spoke 5% similarity    -0.066612          -0.131176   
                 net spoke 95% similarity     0.11777          0.0729009   
                 net spoke PC1              0.0582617          0.0255982   
                 spoke 5% similarity        -0.177564          -0.276488   
                 spoke 95% similarity        0.698852           0.647441   
                 spoke PC1                   0.299288           0.253144   
                 survey 5% similarity       -0.395819          -0.505146   
                 survey 95% similarity       0.711058           0.639585   
                 survey PC1                  0.318089             0.2668   
(dodec, inter)   confidence                     53.38              48.43   
                 consensus                      53.02              51.51   
                 net spoke 5% similarity   -0.0970896           -0.13778   
                 net spoke 95% similarity   0.0967325          0.0439875   
                 net spoke PC1              0.0752066          0.0519753   
                 spoke 5% similarity        -0.245504          -0.324874   
                 spoke 95% similarity        0.646924           0.618184   
                 spoke PC1                   0.269057           0.253164   
                 survey 5% similarity       -0.469243          -0.540434   
                 survey 95% similarity       0.683996             0.6477   
                 survey PC1                  0.342388           0.302235   

                                          Result Upper Bound Effect Size  \
(caveman, indep) confidence                            57.55           0   
                 consensus                             55.15           0   
                 net spoke 5% similarity           -0.131175           0   
                 net spoke 95% similarity            0.31569           0   
                 net spoke PC1                      0.229173           0   
                 spoke 5% similarity               -0.294525           0   
                 spoke 95% similarity               0.900423           0   
             